In [1]:
#import relevant packages for analysis

from __future__ import print_function

import os
import sys

import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

PROJ_ROOT = os.path.join(os.pardir)

sns.set()
sns.set_style("darkgrid")
sns.set_context("poster")

In [2]:
%load_ext watermark
%watermark -a "Bryan Dickinson" -d -t -v -p numpy,pandas

Bryan Dickinson 2019-09-16 18:34:56 

CPython 3.7.3
IPython 7.7.0

numpy 1.16.4
pandas 0.25.0


***Problem Statement***

A retail company “ABC Private Limited” wants to understand the customer purchase behaviour against various products of different categories. They have shared purchase summary of various customers for selected high volume products from last month.
The data set also contains customer demographics (age, gender, marital status, city_type, stay_in_current_city), product details (product_id and product category) and Total purchase_amount from last month.

Now, they want to build a model which will help them to create personalized offer for customers against different products.

**Variable Definition**

**User_ID:** User ID

**Product_ID:**	Product ID

**Gender:**	Sex of User

**Age:** Age in bins

**Occupation:**	Occupation (Masked)

**City_Category:**	Category of the City (A,B,C)

**Stay_In_Current_City_Years:**	Number of years stay in current city

**Marital_Status:**	Marital Status

**Product_Category_1:**	Product Category (Masked)

**Product_Category_2:**	Product may belongs to other category also (Masked)

**Product_Category_3:**	Product may belongs to other category also (Masked)

**Purchase:**	Purchase Amount



### Import the dataset, explore and ready the data for analysis

In [3]:
#Create the path to the data and read into a pandas dataframe

customer_path = os.path.join(PROJ_ROOT, 
                         'data',
                         'train.csv')

data = pd.read_csv(customer_path, 
                 dtype={'User_ID':'category','Product_ID':'category',
                                    'Marital_Status':'category','Gender':'category',
                                    'Age': 'category', 'Occupation':'category', 
                                    'Product_Category_1': 'category',
                                    'Product_Category_2': 'category', 
                                    'Product_Category_3':'category', 
                                    'City_Category': 'category', 
                                    'Stay_In_Current_City_Years': 'category'} )

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
User_ID                       550068 non-null category
Product_ID                    550068 non-null category
Gender                        550068 non-null category
Age                           550068 non-null category
Occupation                    550068 non-null category
City_Category                 550068 non-null category
Stay_In_Current_City_Years    550068 non-null category
Marital_Status                550068 non-null category
Product_Category_1            550068 non-null category
Product_Category_2            376430 non-null category
Product_Category_3            166821 non-null category
Purchase                      550068 non-null int64
dtypes: category(11), int64(1)
memory usage: 11.4 MB


All of these features are categories. 
Alsot, there seems to be some missing values in Product Categories 2 and 3. From the description above, this seems normal. A product can belong to **one or more** categories.

Lets take a look at the columns, and verify there are no anomolies in the naming conventions that could hinder analysis

In [5]:
data.columns

Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3', 'Purchase'],
      dtype='object')

In [6]:
data.Product_ID.nunique()

3631

In [7]:
# get a view of how many labels are in each category
for i in data.columns:
    print(i , data[i].nunique())

User_ID 5891
Product_ID 3631
Gender 2
Age 7
Occupation 21
City_Category 3
Stay_In_Current_City_Years 5
Marital_Status 2
Product_Category_1 20
Product_Category_2 17
Product_Category_3 15
Purchase 18105


In [8]:
data.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6,14,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


It looks like the Product Categories are not sub categories. A product category and be listed in different features here.

#### How much volume was generated during this time frame?

In [9]:
num = data.Purchase.sum()
print('Total volume is: ${:,}'.format(num))

Total volume is: $5,095,812,742


#### What information can we find on customer trips/purchases?

In [10]:
df = data.User_ID.value_counts()
df.describe()

count    5891.000000
mean       93.374300
std       107.190049
min         6.000000
25%        26.000000
50%        54.000000
75%       117.000000
max      1026.000000
Name: User_ID, dtype: float64

The average customer:
- Purchased 93 products over this time period
- The top customer purchased 1,026 products
- There was a wide range of product purchases, from 6 items to 1,026

### Look at the customer data by categories

In [11]:
def fix_df(df):
    ''' A function to help display the pandas series & 
    perform a calculation to total customers for the group by functions'''
    
    #grab the name of the feature the df is looking at
    col = df.columns[0]
    
    # rename the column & index of the new df
    df = df.rename(index = str, columns={col:'count'})
    df.index.names = [col.lower()]
    
    #create a 'total customer' variable
    total_customers = df.sum()
    #create a function to divide groupby segment by the total number of observances
    pct_total = lambda x : x/total_customers
    
    # create a column using the lambda function & return the df
    df['pct'] = df.apply(pct_total, axis = 1)
    return df

**What is the predominate age group that shopped?**

In [12]:
#pass the original df with the groupby function to fix_df
fix_df(pd.DataFrame(data.Age.value_counts(dropna=False)))

,count,pct
age,,
26-35,219587,0.399200
36-45,110013,0.199999
18-25,99660,0.181178
46-50,45701,0.083082
51-55,38501,0.069993
55+,21504,0.039093
0-17,15102,0.027455


The top purchasing age group was aged 26-35, they accounted for 40% of customers

**What is the gender demographic of customers?**

In [13]:
fix_df(pd.DataFrame(data.Gender.value_counts(dropna=False)))

,count,pct
gender,,
M,414259,0.753105
F,135809,0.246895


3 of 4 customers for ABC Retail company is male

**Which gender spends more per purchase on average?**

In [14]:
pd.DataFrame(data.groupby(['Gender'])['Purchase'].mean())

,Purchase
Gender,
F,8734.565765
M,9437.526040


Males not only make up the majority of purchasing  customers, they also spend $703 more on average than females

**Is there a shopping difference among occupations?**

In [15]:
fix_df(pd.DataFrame(data.Occupation.value_counts(dropna=False))).head(10)

,count,pct
occupation,,
4,72308,0.131453
0,69638,0.126599
7,59133,0.107501
1,47426,0.086218
17,40043,0.072796
20,33562,0.061014
12,31179,0.056682
14,27309,0.049647
2,26588,0.048336


The top three occupations account for 37% of their customers

**Do the top 5 occupations contibute the same volume compared to their trips?**

In [16]:
fix_df(pd.DataFrame(data.groupby('Occupation')['Purchase'].sum().sort_values(ascending = False))).head(10)

,count,pct
purchase,,
4,666244484,0.130744
0,635406958,0.124692
7,557371587,0.109378
1,424614144,0.083326
17,393281453,0.077177
12,305449446,0.059941
20,296570442,0.058199
14,259454692,0.050915
16,238346955,0.046773


Most of occupations' spend match the frequency of these occupations, which means that the average spend accross occupations is rather similar.

**Martial status of shoppers**

In [17]:
fix_df(pd.DataFrame(data.Marital_Status.value_counts(dropna=False)))

,count,pct
marital_status,,
0,324731,0.590347
1,225337,0.409653


60% of shoppers are single

**Is there a spend difference by Marital status?**

In [18]:
pd.DataFrame(data.groupby('Marital_Status')['Purchase'].mean())

,Purchase
Marital_Status,
0,9265.907619
1,9261.174574


No, both single and married customers spend similar amounts

**In which City Category do our customers live?**

In [19]:
data.City_Category.unique()

[A, C, B]
Categories (3, object): [A, C, B]

In [20]:
fix_df(pd.DataFrame(data.City_Category.value_counts(dropna=False)))

,count,pct
city_category,,
B,231173,0.420263
C,171175,0.311189
A,147720,0.268549


**Does spend vary by city?**

In [21]:
pd.DataFrame(data.groupby('City_Category')[['Purchase']].mean())

,Purchase
City_Category,
A,8911.939216
B,9151.300563
C,9719.920993


There is a larger spend in City_Category C

**Gain more information on the spends of our customers**

In [22]:
data.Purchase.describe()

count    550068.000000
mean       9263.968713
std        5023.065394
min          12.000000
25%        5823.000000
50%        8047.000000
75%       12054.000000
max       23961.000000
Name: Purchase, dtype: float64

In [23]:
#90th Percentile of Purchases
np.percentile(data.Purchase, 90)

16317.0

**Who are the top customers by number of purchases?**

In [24]:
df_u = pd.DataFrame( data.groupby('User_ID').size().sort_values(ascending = False), columns = ['# of products purchased'])
df_u.head(10)

,# of products purchased
User_ID,
1001680,1026
1004277,979
1001941,898
1001181,862
1000889,823
1003618,767
1001150,752
1001015,740
1005795,729


**Look at purchases by categories**

In [25]:
# grab the mean of the purchases
p_mean = data.Purchase.mean()

**Total purchases by customer age group**

In [26]:
#Group the dataframe by Age and sum the Purchases
df = data.groupby('Age')[['Purchase']].sum()

# rename the column & index of the new df
col = df.columns[0]
df = df.rename(index = str, columns={col:'sum'})
df.index.names = [col.lower()]


#create a 'total customer' variable, the total spend among all customers in the df
total_customers = df.sum()


#create a function to divide groupby segment (Age bins) by the total number of observances
pct_total = lambda x : x/total_customers
    
# create a column using the lambda function & return the df
df['pct'] = df.apply(pct_total, axis = 1)
df


,sum,pct
purchase,,
0-17,134913183,0.026475
18-25,913848675,0.179333
26-35,2031770578,0.398714
36-45,1026569884,0.201454
46-50,420843403,0.082586
51-55,367099644,0.072039
55+,200767375,0.039398


Age group 26-35 is the highest spending of customers followed by 36-45.

**Are there any insights on the category purchases by age group?**

In [27]:
#Which age groups purchased the most items?
df = pd.DataFrame(data.groupby('Age').size().sort_index()).reset_index()
df = df.rename(columns = {0:'Count'})
df

,Age,Count
0,0-17,15102
1,18-25,99660
2,26-35,219587
3,36-45,110013
4,46-50,45701
5,51-55,38501
6,55+,21504


In [28]:
data.groupby('Age')[['Product_Category_1','Product_Category_2', 'Product_Category_3']].count()

,Product_Category_1,Product_Category_2,Product_Category_3
Age,,,
0-17,15102,10648,4873
18-25,99660,69157,31316
26-35,219587,150160,66942
36-45,110013,75350,33285
46-50,45701,31010,13374
51-55,38501,26024,11166
55+,21504,14081,5865


In [29]:

#Group the category columns by Age, and count how many each age group has purchased
df = data.groupby('Age')[['Product_Category_1','Product_Category_2', 'Product_Category_3']].count()

#create a function to find the sum of a row, and divide each cell by the total 
#     for a % of product category purchases by age group

def cat_1p(row):
    ttl = row.sum()
    return row['Product_Category_1']/ ttl 
def cat_2p(row):
    ttl = row.sum()
    return row['Product_Category_2']/ ttl 
def cat_3p(row):
    ttl = row.sum()
    return row['Product_Category_3']/ ttl 

# change the index type of the df and append a total row at the bottom of the df to show product category totals
df.index = df.index.astype(object)
df = df.append(pd.DataFrame(df.apply(np.sum, axis = 0)).transpose().rename({0:'Total'}, axis = 'index'))


#execute the category percentages and place them in new columns in the df
df['Categ_1_Pct'] = df.apply(cat_1p, axis = 1)
df['Categ_2_Pct'] = df.apply(cat_2p, axis = 1)
df['Categ_3_Pct'] = df.apply(cat_3p, axis = 1)

#rearrange the df to better read the percentage next to the corresponding category
df = df[['Product_Category_1', 'Categ_1_Pct','Product_Category_2', 'Categ_2_Pct','Product_Category_3', 'Categ_3_Pct']]


df

,Product_Category_1,Categ_1_Pct,Product_Category_2,Categ_2_Pct,Product_Category_3,Categ_3_Pct
0-17,15102,0.493159,10648,0.347707,4873,0.159124
18-25,99660,0.497969,69157,0.345554,31316,0.156475
26-35,219587,0.502845,150160,0.343860,66942,0.153294
36-45,110013,0.503151,75350,0.344617,33285,0.152230
46-50,45701,0.507310,31010,0.344229,13374,0.148458
51-55,38501,0.508660,26024,0.343817,11166,0.147519
55+,21504,0.518794,14081,0.339706,5865,0.141493
Total,550068,0.503118,376430,0.344300,166821,0.152582


Product Category 1 has 50 % of the purchases, follwed by categories 2 and 3 respectivley. Each age group follows this same trend.
I'm not sure how much information this really tells us. We know that products can be in more than one category. This shows us that the majority of products only have 1 (product category 1).

**Lets take a look at the average spend by age group**

In [30]:
#Group the data by Age, then take the mean of the Purchase feature
df =  data.groupby('Age')[['Purchase']].mean()

#Add take the mean of the df and transpose the df to add it to the bottom.
df.index = df.index.astype(object)
df = df.append(pd.DataFrame(df.apply(np.mean, axis = 0)).transpose().rename({0:'Total'}, axis = 'index'))
df

,Purchase
0-17,8933.464640
18-25,9169.663606
26-35,9252.690633
36-45,9331.350695
46-50,9208.625697
51-55,9534.808031
55+,9336.280459
Total,9252.411966


Average total purchases are similar across age groups. We see the highest purchase is $9,336 from 51-55 age group and the lowest from the 0-17 age group. 

In [31]:


#Group the category columns by Age, and count how many each age group has purchased
df = data.groupby('Gender')[['Product_Category_1','Product_Category_2', 'Product_Category_3']].count()

# change the index type of the df and append a total row at the bottom of the df to show product category totals
df.index = df.index.astype(object)
df = df.append(pd.DataFrame(df.apply(np.sum, axis = 0)).transpose().rename({0:'Total'}, axis = 'index'))


#execute the category percentages and place them in new columns in the df
df['Categ_1_Pct'] = df.apply(cat_1p, axis = 1)
df['Categ_2_Pct'] = df.apply(cat_2p, axis = 1)
df['Categ_3_Pct'] = df.apply(cat_3p, axis = 1)

#rearrange the df to better read the percentage next to the corresponding category
df = df[['Product_Category_1', 'Categ_1_Pct','Product_Category_2', 'Categ_2_Pct','Product_Category_3', 'Categ_3_Pct']]


df

,Product_Category_1,Categ_1_Pct,Product_Category_2,Categ_2_Pct,Product_Category_3,Categ_3_Pct
F,135809,0.512616,91530,0.345483,37594,0.141900
M,414259,0.500080,284900,0.343922,129227,0.155998
Total,550068,0.503118,376430,0.344300,166821,0.152582


Product Categories are very much similar across product categories

**What is the volume difference among Marital Status?**

In [32]:

def change_cur(row):
    """Function to format the Purchase column to formatted currency"""
    
    # replace the number in the Purchase column with commas & a '$' sign.
    row['Purchase'] =' ${:,}'.format(row['Purchase'])
    
    #return the new, formatted value
    return row['Purchase']

In [33]:
df = data.groupby('Gender')[['Purchase']].sum()

#apply the function to the Purchase column using the apply method
df['Purchase'] = df.apply(change_cur, axis = 1)

df

,Purchase
Gender,
F,"$1,186,232,642"
M,"$3,909,580,100"


What if we also group by Age?

In [34]:
df = data.groupby(['Age','Gender'])[['Purchase']].sum()
df['Purchase'] = df.apply(change_cur, axis = 1)
df = df.groupby(['Age','Gender'])[['Purchase']].sum().unstack(0)
df

Purchase                                                 \
Age             0-17          18-25            26-35          36-45   
Gender                                                                
F        $42,385,978   $205,475,842     $442,976,233   $243,438,963   
M        $92,527,205   $708,372,833   $1,588,794,345   $783,130,921   

                                                     
Age             46-50          51-55            55+  
Gender                                               
F        $116,706,864    $89,465,997    $45,782,765  
M        $304,136,539   $277,633,647   $154,984,610

In [35]:
#create the path for the new file
customer_data_clean = os.path.join(PROJ_ROOT, 
                         'data', 'processed',
                         'clean_data.csv')

data.to_csv(customer_data_clean, index = False)